In [122]:
using LinearAlgebra
using Plots
gr()


function power_method(A, x, tol=1.0e-8)    
    n = 0
    mu = undef
    err_plot = []
    err_n = []
    errs = [1.0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6, 1.0e-7, 1.0e-8]

    while true
        x0 = x
        xn = A*normalize(x)
        x = normalize(xn)
        mu = x'*xn

        n += 1
        err = norm(x0-x, Inf)
        push!(err_plot, err)

        if err < errs[1]
            push!(err_n, n)
            popfirst!(errs)
        end

        if err < tol
            break
        end
    end

    println("eigenvector: $(x)\neigenvalue: $(mu)\niterations: $(n)")

    err_n_plot = [err_plot[i] for i in err_n]
    plot(collect(1:n), err_plot, yaxis=:log, lw=4, lc=:teal, label="",
        yticks=[1.0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6, 1.0e-7, 1.0e-8],
        xticks=[i for i in 0:5:n],
        ylabel="absolute error", xlabel="iterations", size=(900,900),
        title="Maximum eigenvalue error vs number of iterations")
    scatter!(err_n, err_n_plot, mc=:gold, ms=6, label="")
    png("power method $(n) iterations")
end


function qr_decomp(A, tol=1.0e-8)
    n = 0
    err_plot = []
    err_n = []
    errs = [1.0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6, 1.0e-7, 1.0e-8]

    while true
        A = qr(A)
        A = A.R*A.Q
        
        n += 1
        err = norm(diag(A, -1), 1)
        push!(err_plot, err)

        if err < errs[1]
            push!(err_n, n)
            popfirst!(errs)
        end

        if err < tol
            break
        end
    end

    println("iterations: $(n)\neigenvalues:")
    for e in sort(diag(A))
        println("    $(e)")
    end

    err_n_plot = [err_plot[i] for i in err_n]
    plot(collect(1:n), err_plot, yaxis=:log, lw=4, lc=:teal, label="",
        yticks=[1.0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5, 1.0e-6, 1.0e-7, 1.0e-8],
        xticks=[i for i in 0:5:n],
        ylabel="absolute error", xlabel="iterations", size=(900,900),
        title="Off-diagonal error vs number of iterations")
    scatter!(err_n, err_n_plot, mc=:gold, ms=6, label="")
    png("qr $(n) iterations")
end


function shift_qr(A, tol=1.0e-8)
    n = 0
    eigs = []
    err_plot = []

    while true
        K = A[end,end]*I(size(A,1))
        A = qr(A - K)
        A = A.R*A.Q + K
        n += 1
        
        if size(A, 1) == 1
            push!(eigs, A[end,end])
            push!(err_plot, 1.0e-8)
            break
        end

        err = abs(A[end-1,end])
        push!(err_plot, err)

        if (abs(A[end-1,end]) < tol) & (abs(A[end,end-1]) < tol)
            push!(eigs, A[end,end])
            A = A[1:end-1, 1:end-1]
        end
    end

    println("iterations: $(n)\neigenvalues:")
    for e in sort(eigs)
        println("    $(e)")
    end

    plot(collect(1:n), err_plot, yaxis=:log, lw=4, lc=:teal, label="",
        ylabel="absolute error", xlabel="iterations", size=(900,900),
        title="QR+shift off-diagonal error vs number of iterations")
    scatter!(collect(1:n), err_plot, mc=:gold, ms=6, label="")
    
    png("shift qr $(n) iterations")
end;

In [123]:
A = [4. 2. 1.; 0. 3. 2.; 1. 1. 4.]
x = [1., 2., 1.]

power_method(A, x)

eigenvector: [0.7135786481604682, 0.3959205712924121, 0.5779726759259203]
eigenvalue: 5.919639578909249
iterations: 23


In [124]:
A = [4. 1. 1. 1.; 1. 3. -1. 1.; 1. -1. 2. 0.; 1. 1. 0. 2.]
x = [1., -2., 0., 3.]

power_method(A, x)

eigenvector: [0.7794684516923143, 0.48173797465578533, 0.09200378498008333, 0.38973421730747093]
eigenvalue: 5.236067977499789
iterations: 48


In [125]:
A = [4. -1. 0.; -1. 3. -1.; 0. -1. 2.]
qr_decomp(A)

iterations: 41
eigenvalues:
    1.267949192431124
    2.9999999999999956
    4.732050807568875


In [126]:
A = [-2. 1. 0. 0.; 1. -3. -1. 0.; 0. -1. 1. 1.; 0. 0. 1. 3.]

qr_decomp(A)

iterations: 178
eigenvalues:
    -3.7782865121039415
    -1.4880677957482995
    0.8275516854923803
    3.4388026223598525


In [127]:
A = [4. -1. 0.; -1. 3. -1.; 0. -1. 2.]

shift_qr(A)

iterations: 8
eigenvalues:
    1.2679491924311226
    3.0000000000000004
    4.732050807568877


In [121]:
A = [-2. 1. 0. 0.; 1. -3. -1. 0.; 0. -1. 1. 1.; 0. 0. 1. 3.]

shift_qr(A)

iterations: 9
eigenvalues:
    -3.778286512103936
    -1.4880677957483008
    0.827551685492379
    3.438802622359853
